# Company Web Scraping through Bing

Import all necessary packages for web scraping

In [1]:
import time
import csv
import os
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Graduate\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Conduct Web Scraping of Company Names

In [ ]:
# Load the DataFrame with company and description columns
df = pd.read_csv("C:/Users/Graduate/Desktop/Grad MSBA/Advanced Python/Final Project/Test Dataset/company_info_final - company_info_final.csv")

# Load the NAICS_subcategories file
naics_subcategories = pd.read_csv("NAICS_subcategories.csv")

# Merge the two DataFrames on the first 3 digits of the NAICS code
df = pd.merge(df, naics_subcategories, left_on=df["NAICS on SoS site"].astype(str).str[:3], right_on=naics_subcategories["NAICS Code"].astype(str).str[:3], how="left")

# Define an empty dictionary to store the results
keyword_dict = {}

# Add keyword_count and found_count columns to the DataFrame
df["keyword_count"] = 0
df["found_count"] = 0

# Add a "Manufacturing" column to the DataFrame and set it to False
df["Manufacturing"] = False

# Create a new instance of the Firefox driver
driver = webdriver.Firefox()

# Define a set of stopwords
stop_words = set(stopwords.words('english'))

# Loop through all rows in the DataFrame
for index, row in df.iterrows():
    # Check if the NAICS Code is blank
    if pd.isnull(row["NAICS on SoS site"]):
        continue  # skip to next row
    
    company = row["Company"].strip()
    description = row["Description"]
    
    company_name = company.replace("&", "and")
        
    # Navigate to bing.com with the search query
    driver.get("https://www.bing.com/search?q=" + company_name + " " + "manufacturing" + " " + "Rhode Island")
        
    # Wait for the search results to load
    driver.implicitly_wait(10)
        
    # Find the first search result link that is not an advertisement
    search_results = driver.find_elements('css selector', "div.tF2Cxc")
    for result in search_results:
        try:
            search_link = result.find_element('tag name', 'a')
            if 'http' in search_link.get_attribute('href') and 'google' not in search_link.get_attribute('href'):
                search_link.click()
                break
        except:
            pass
        
    # Wait for the resulting page to load
    driver.implicitly_wait(10)
        
    # Check if the description is blank
    if pd.isnull(description):
        df.at[index, "Manufacturing"] = False
        continue
        
    # Get the webpage content
    webpage = driver.page_source
        
    # Loop through each keyword for the current company
    for word in description.split():        
        if word.lower() not in stop_words:
            keyword_dict.setdefault(word.lower(), []).append(1 if word.lower() in webpage.lower() else 0)
            # Increment the found count for the current row if the keyword is found in the webpage
            df.at[index, "found_count"] += 1 if word.lower() in webpage.lower() else 0
            # Increment the keyword count for the current row
            df.at[index, "keyword_count"] += 1
        else:
            keyword_dict.setdefault(word.lower(), []).append(0)

        if word.lower() == 'manufacturing':
            df.at[index, "Manufacturing"] = True

    # Add the found count to the DataFrame
    df.at[index, "found_count"] = df.at[index, "found_count"]
    df.at[index, "Manufacturing"] = df.at[index, "Manufacturing"]
        
    # Calculate the percentage of keywords found and add to the DataFrame
    percentage = df.at[index, "found_count"] / df.at[index, "keyword_count"] * 100 if df.at[index, "keyword_count"] > 0 else 0
    df.at[index, "found_percentage"] = "{:.2f}%".format(percentage)
        
# Close the web driver
driver.quit()

Generate Excel file and place it in the "Web Scrape" folder

In [3]:
# Create the folder if it doesn't exist
if not os.path.exists("Web Scrape"):
    os.makedirs("Web Scrape")

# drop the first column
df = df.drop(df.columns[0], axis=1)

# Save the updated DataFrame to a new CSV file in the Web Scrape folder
df.to_csv("Web Scrape/bing_web_scrape.csv", index=False)

NameError: name 'df' is not defined